In [1]:
import duckdb as dd
from pathlib import Path

### Staging Tabelle

In [2]:
connect = dd.connect("data\\warehouse.duckdb")

In [9]:
connect.execute(
    "CREATE TABLE IF NOT EXISTS staging_data AS " \
    "SELECT * FROM read_csv_auto('data\\example.csv')")

In [10]:
connect.sql("select * from staging_data limit 5")

┌─────────────────────┬─────────────┬────────────┬───────────┬─────────┬────────────┬───────────────┬───────────────────┬────────────────────┬────────┬────────────────┬──────────────┬───────────────────────┐
│    Purchase date    │ Customer ID │ First_Name │ Last_Name │ Gender  │  Birthday  │ Support Level │       City        │    Servicename     │ Costs  │ payment_method │ Sales Canal  │ Customer Satisfaction │
│      timestamp      │    int64    │  varchar   │  varchar  │ varchar │    date    │    varchar    │      varchar      │      varchar       │ double │    varchar     │   varchar    │         int64         │
├─────────────────────┼─────────────┼────────────┼───────────┼─────────┼────────────┼───────────────┼───────────────────┼────────────────────┼────────┼────────────────┼──────────────┼───────────────────────┤
│ 2024-06-06 19:13:54 │  4152186239 │ Nicholas   │ Avila     │ M       │ 1992-05-30 │ Premium       │ North Danielton   │ Custom Development │ 199.99 │ PayPal         │

In [11]:
connect.sql("describe select * from staging_data limit 5")

┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │ column_type │  null   │   key   │ default │  extra  │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Purchase date         │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ Customer ID           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ First_Name            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Last_Name             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Gender                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Birthday              │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ Support Level         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ City                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Servicename           │ VARCHAR     │ 

### Erste kleine Analysen

In [20]:
# Wie viele unterschiedliche Kunden gibt es?
query = """
SELECT 
  count(DISTINCT "Customer ID") as Anzahl
FROM staging_data  
"""
connect.sql(query)

┌────────┐
│ Anzahl │
│ int64  │
├────────┤
│    100 │
└────────┘